## Integrating 2D functions by avoiding singularity ## 

**The code below is partly from scipy.quad library and I have used 3-4 _Stack Overflow_ responses. I have lost the _Stack Overflow_ sources, so I am not able to cite them. All sources that I used were license-free for modifications.** 

_I will still not call these as my codes._

The codes below integrates 1D functions by avoiding the the singularity area (poles area). Codes provided in Python library `scipy.integrate` `quad` gives an `Exception` error when singularity are encountered. 
The avoidance of singularity area contributes to error in integral computation. This can be minimized by using higher level of precision, i.e., reducing the area where singularity exists. The information on the location of singularity points are not a prerequisite of the developed code.

The execution of code is quite fast for 1D functions. Higher level precision >100 can be attempted for initial trials.

The main code is followed by an example, which can be modified. 

**The required libraries**

In [93]:
import numpy as np
from tqdm import tqdm

**The modified exception error class.**

In [77]:
class Int_Error(Exception):
    
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return self.message
        return "Error Message"

## The main code ##

We first define the **_Int_pole_** `class` and then define the `function` **_int1D_**.


In [86]:
class Int_pole:
    
    def __init__(self, function):
        self.function = function
        self.error = 0
        self.sign = 1

# you can change the precision to higher value for better accuracy

    def int1D(self, lower, upper, precision):
        
        if lower > upper:
            lower, upper = upper, lower
            self.sign = -1  # forces change in limits direction and the sign.
            
        num_points = (upper - lower) * precision
        xs = np.linspace(lower, upper, int(num_points))
        integral = 0
        sup_sum = 0
        sub_sum = 0
        
        for index in tqdm(range(len(xs) - 1)):
            delta = xs[index + 1] - xs[index]
            try:
                y1 = self.function(xs[index])
                sub_area = y1 * delta
                y2 = self.function(xs[index + 1])
                sup_area = y2 * delta

                area = (y2 + y1) / 2 * delta
                integral += area
                sub_sum += sub_area
                sup_sum += sup_area
                
            except ZeroDivisionError: # avoiding error
                print(f"\n Avoided pole")

        self.error = sup_sum - sub_sum
        return self.sign * integral

**Let us check the code using the following test function which has a pole at $x=0$:**

$$
\sin(x)/x
$$

**_Note_**: Make the precision value sufficiently high, maybe greater than **100** to start with.

In [95]:
def test_function(x):
    return np.sin(x)/x

In [97]:
integral = Int_pole(test_function)
result = integral.int1D(-1, 1, precision=10000)
print(result)
print("\nThe accuracy of this result is", integral.error)

100%|██████████| 19999/19999 [00:00<00:00, 175351.95it/s]

1.8921661402323633

The accuracy of this result is 2.220446049250313e-16


**We check the 1D function here using Standard Python library `scipy.quad` `function` `quad`.
Check the [link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad).**

In the example we integrate using `quad` the same test function as we did above with our code.

In [99]:
from scipy import integrate
x2 = lambda x: np.sin(x)/x
integrate.quad(x2, -1, 1)

<ipython-input-99-633eaf70919a>:2: RuntimeWarning: invalid value encountered in double_scalars
  x2 = lambda x: np.sin(x)/x
<ipython-input-99-633eaf70919a>:3: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integrate.quad(x2, -1, 1)


(nan, nan)